In [5]:
%cd ~/desktop/RNA/casp16
!cat ~/.bashrc | grep -i "export"

/Users/sumishunsuke/Desktop/RNA/casp16
export DFIRE_RNA_HOME=/Users/sumishunsuke/Desktop/RNA/casp16/third_party/dfire_rna
export BRiQ_DATAPATH=~/desktop/RNA/casp16/third_party/RNA-BRiQ/BRiQ_data
export PATH=$PATH:~/desktop/RNA/casp16/third_party/RNA-BRiQ/build/bin


# R1209のenergy計算

In [6]:
ls ./datasets/casp16/R1209/pdb/*pdb | wc -l

    1000


In [7]:
pdb_dir="./datasets/casp16/R1209/pdb"
!python ./scripts/eval_scores.py --pdb_dir {pdb_dir} --out_dir {pdb_dir}

['S_000668.pdb', 'S_000654.pdb', 'S_000132.pdb', 'S_000126.pdb']
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/sumishunsuke/micromamba/envs/casp16/lib/python3.12/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/sumishunsuke/micromamba/envs/casp16/lib/python3.12/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/Users/sumishunsuke/Desktop/RNA/casp16/./src/score/RNA_BRiQ.py", line 50, in process_pdb_file_eval
    briq.AssignSS()
  File "/Users/sumishunsuke/Desktop/RNA/casp16/./src/score/RNA_BRiQ.py", line 20, in AssignSS
    with open(self.briq_input, "r") as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './datasets/casp16/R1209/pdb/S_000668.briq.in'
"""

The above exception was the direct cause of the following exception:

Traceback (most rec

# R1209の距離計算, MDS